# MICOM model Construction & simulation

* This jupyter is for constructing community models for genus/species level, and simulating it.
* Please take into note for this for further info https://micom-dev.github.io/micom/index.html 
* Make sure to install the packages versions as listed below. I recommend just using the virtual environment [VENV-390_micom]

In [1]:
#install packages
# !pip install openpyxl==3.1.5
# !pip install micom==0.37.1
# !pip install pandas==2.3.3
# !pip install cobra==0.29.1
# !pip install seaborn


#import packages
import pandas as pd
import pickle,  cobra #openpyxl,
import re, micom
from micom.workflows import grow
from micom import Community, data
from micom import load_pickle
import warnings
import seaborn
warnings.filterwarnings("ignore")

### Import and Export data
* Abundance file : The observed taxon abundance data. Have to conform to the csv format provided as example.
* GEM database : Use genus GEM db for genus level, use species db for species. Download at https://zenodo.org/records/3755182
* Dietary info | Need a dataframe of reaction, upper bound flux (flux). Recommend just using the provided qza, https://github.com/micom-dev/media?tab=readme-ov-file



* various output data

In [2]:
#----Taxon abundance data input----
abundance_data_directory =  "yunusbayev_genus.csv"  #Input here
abundance_data = pd.read_csv("input/" + abundance_data_directory)

#Phenotype (control & healthy)
metadata = pd.read_csv("input/"+ "yunusbayev_metadata.csv")  #Input here
phenotype = metadata["host_disease"]
phenotype.index = metadata["sample"]

#---- Model database input ----
db_dir = "database/" + "agora201_refseq216_genus.qza" #Input here

#---- dietary information input ----
from micom.qiime_formats import load_qiime_medium
medium = load_qiime_medium("database/"+"western_diet_gut.qza")  #Input here



#---- Model simulation parameters ----
trade_off = 0.5
cut_off = 0.001  
type = "FBA"
solver = "gurobi"
cpu_threads = 4

#output directory, don't change below
model_name = abundance_data_directory.split(".")[0] +"_"+ str(cut_off)
output_name = f"{model_name}_{type}_{trade_off}"

## Constructing the model
* We just pass our taxonomy table and model database to MICOM. 
*  remove all taxa that make up less than cutoff % of the community 

In [ ]:
from micom.workflows import build
from micom import Community
import pandas as pd

manifest = build(abundance_data, db_dir, f"models/{model_name}", solver=solver, cutoff=cut_off, threads=cpu_threads)

## Community Model Growth Simulation
* simulation can take a long, better to save the results as pickle 
* For all samples simulation, use "grow(), make sure to use pFBA or the flux would show unreasonable high flux
* Increasing tradeoff (→1.0) pushes the community toward a single “maximize total growth” optimizer.
* The optimal cutoff =  highest trade-off where the maximal number of taxa were enabled to grow

In [4]:
# Run the simulation
if type == "FBA":
    growth_results = grow(manifest, f"models/{model_name}", medium, tradeoff=trade_off, threads=cpu_threads)
else:
    growth_results = grow(manifest, f"models/{model_name}", medium, tradeoff=trade_off, threads=cpu_threads,strategy="pFBA")


#check for missing gr, because if there is abundance, taxon should be able to grow!!!!!!!
found_gr = set(list(growth_results.growth_rates["sample_id"].unique()))
all_gr = set(list(metadata["sample"]))
print("samples not included are... " ,all_gr - found_gr)
print("any taxons with zero growth...  ",(growth_results.growth_rates["growth_rate"] == 0).any())


# Save the results to a pickle file
pickle.dump(growth_results, open("growth_pickle/" + output_name+ ".pickle", "wb"))


# Save the growth results to excel file
with pd.ExcelWriter("results/"+output_name+ "_results.xlsx") as writer:
    growth_results.growth_rates.to_excel(writer, sheet_name="growth_rates", index=True)
    growth_results.exchanges.to_excel(writer, sheet_name="exchanges", index=False)
    growth_results.annotations.to_excel(writer, sheet_name="annotation", index=True)

samples not included are...  set()
any taxons with zero growth...   False


In [3]:
#Load the previous simulation
growth_results = pickle.load(open("growth_pickle/" + output_name+ "_growth.pickle", "rb"))

## Individual subject simulation
* When you want to simulate a single subject's microbiome community.
* Ex) only simulate D01_profile's microbiome and predict the metabolic fluxes.
* By default micom assigns the community growth rate as the objective for a community model.

In [ ]:
#input the subject's name
indiv_subject_name = "D01_profile"


indiv_model = load_pickle(f"models/{model_name}/{indiv_subject_name}.pickle")


#load and fix the default medium
medium_dict = medium["flux"].to_dict()
indiv_model.medium = medium_dict


#perform simulation
indiv_model_sim = indiv_model.cooperative_tradeoff(fraction=0.5 , fluxes=True, pfba=True)

#export as xlsx
with pd.ExcelWriter(f"results/{model_name}_{indiv_subject_name}_results.xlsx") as writer:
    indiv_model_sim.members.to_excel(writer, sheet_name="members", index=True)
    indiv_model_sim.fluxes.to_excel(writer, sheet_name="Fluxes", index=True)